In [ ]:
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [ ]:

# Configura Selenium para usar Chrome
service = Service(ChromeDriverManager().install())
# Dont open the browser
options = webdriver.ChromeOptions()
options.add_argument("--headless") 
# Driver
driver = webdriver.Chrome(service=service,options=options)


user = 'ComputingVictor'
# Abre la URL objetivo
driver.get(f'https://www.kaggle.com/{user}')

# Usa una espera implícita si es necesario para asegurar que la página haya cargado
driver.implicitly_wait(10)

# Encuentra el elemento por su atributo data-testid
elemento = driver.find_element(By.XPATH, '/html/body/main/div[1]/div/div[5]/div[2]/div[2]/div/div/div[5]/div[1]/div[1]')

# Utilizar expresiones regulares para extraer la información relevante
patron_medallas = re.compile(r"(Competitions|Datasets|Notebooks|Discussions)\n([A-Za-z]+)\nMEDALS\n(?:none yet|(\d+)(?:\n(\d+)(?:\n(\d+))?)?)")

# Extraer todas las coincidencias
matches = patron_medallas.findall(elemento.text)

# Convertir los matches en un diccionario adecuado para la creación de un DataFrame
datos_nuevos = {match[0]: {"Level": match[1], "Gold": 0, "Silver": 0, "Bronze": 0} for match in matches}

# Cierra el navegador
driver.quit()

# Actualizar el diccionario con la información de las medallas, ajustando según la cantidad de números encontrados
for match in matches:
    category = match[0]
    # Convertir valores de medallas a entero, usando 0 para valores faltantes
    medallas = [int(x) if x.isdigit() else 0 for x in match[2:]]
    print(medallas)
    if len(medallas) == 3:  # Oro, Plata, Bronce
        datos_nuevos[category]["Gold"], datos_nuevos[category]["Silver"], datos_nuevos[category]["Bronze"] = medallas
    elif len(medallas) == 2:  # Plata, Bronce
        datos_nuevos[category]["Silver"], datos_nuevos[category]["Bronze"] = medallas
    elif len(medallas) == 1:  # Bronce
        datos_nuevos[category]["Bronze"] = medallas[0]

# Reajustar la lógica para cumplir con el requisito exacto del usuario
for category, details in datos_nuevos.items():
    gold, silver, bronze = details["Gold"], details["Silver"], details["Bronze"]
    
    # Si bronze es 0 y hay medallas en oro o plata, deslizar las medallas hacia la derecha
    if bronze == 0 and silver == 0 :
        # Deslizar las medallas a la derecha correctamente
        details["Bronze"], details["Silver"], details["Gold"] = gold, 0, 0
    elif bronze == 0 and silver > 0:
        # Deslizar las medallas a la derecha correctamente
        details["Bronze"], details["Silver"], details["Gold"] = silver, gold, 0

# Convertir el diccionario ajustado a DataFrame para mostrar los resultados
df_ajustado_final = pd.DataFrame(datos_nuevos).T

df_ajustado_final
